# Example of KEO Taylor series expansion for methane

Here we use the following internal coordinates

1. $r_1$
2. $r_2$
3. $r_3$
4. $r_4$
5. $s_1 = (2\alpha_{12} - \alpha_{13} - \alpha_{14} - \alpha_{23} - \alpha_{24} + 2\alpha_{34})/\sqrt{12}$
6. $s_2=\alpha_{13} - \alpha_{14} - \alpha_{23} + \alpha_{24}) / 2$
7. $s_3=(\alpha_{24} - \alpha_{13}) / \sqrt{2}$
8. $s_4=(\alpha_{23} - \alpha_{14}) / \sqrt{2}$
9. $s_5=(\alpha_{34} - \alpha_{12}) / \sqrt{2}$,

where $r_i\equiv\text{C-H}_i$ and $\alpha_{ij}\equiv\text{H}_i\text{-C-H}_j$

In [1]:
import itertools

import jax
import jax.numpy as jnp
import numpy as np
from scipy.special import factorial

from vibrojet._keo import G_to_invcm, Gmat, com, eckart

jax.config.update("jax_enable_x64", True)

First, define a function to obtain five (six) $\alpha_{ij}$ valence angular coordinates from the five symmetrized $s_k$ coordinates.

In [2]:
def from_symmetrized_alpha_to_alpha(symm_alpha, alpha_ini, no_iter: int = 10):
    """Gradient-descend approach for obtaining six angular coordinates 'alpha'
    from five symmetrized combinations of 'alpha' coordinates
    """

    def symmetrized_alpha(alpha):
        alpha12, alpha13, alpha14, alpha23, alpha24 = alpha

        cosbeta = (jnp.cos(alpha23) - jnp.cos(alpha12) * jnp.cos(alpha13)) / (
            jnp.sin(alpha12) * jnp.sin(alpha13)
        )
        beta312 = jnp.acos(cosbeta)

        cosbeta = (jnp.cos(alpha24) - jnp.cos(alpha12) * jnp.cos(alpha14)) / (
            jnp.sin(alpha12) * jnp.sin(alpha14)
        )
        beta412 = jnp.acos(cosbeta)

        cosa34 = jnp.cos(alpha13) * jnp.cos(alpha14) + jnp.cos(
            beta312 + beta412
        ) * jnp.sin(alpha13) * jnp.sin(alpha14)
        alpha34 = jnp.acos(cosa34)

        sym = jnp.array(
            [
                (2 * alpha12 - alpha13 - alpha14 - alpha23 - alpha24 + 2 * alpha34)
                / jnp.sqrt(12),
                (alpha13 - alpha14 - alpha23 + alpha24) / 2,
                (alpha24 - alpha13) / jnp.sqrt(2),
                (alpha23 - alpha14) / jnp.sqrt(2),
                (alpha34 - alpha12) / jnp.sqrt(2),
            ]
        )
        return sym, alpha34

    alpha = alpha_ini

    for _ in range(no_iter):
        sym, alpha34 = symmetrized_alpha(alpha)
        jac = jax.jacrev(symmetrized_alpha)(alpha)[0]
        am = jac.T @ jac
        ai = jnp.linalg.inv(am)
        bm = (symm_alpha - sym) @ jac
        alpha = alpha + ai @ bm

    stdev = jnp.sqrt(jnp.mean(jnp.square(symm_alpha - sym)))
    return jnp.append(alpha, jnp.array([alpha34])), stdev

Define a function to compute the Cartesian coordinates of atoms from given internal coordinates. This function also converts the five symmetrized angular coordinates into six angular coordinates using the function above.

In [17]:
masses = [
    12.0,
    1.007825035,
    1.007825035,
    1.007825035,
    1.007825035,
]  # masses of C, H1, H2, H3, H4

r_ref = 1.0859638364
x0 = np.array([r_ref, r_ref, r_ref, r_ref, 0.0, 0.0, 0.0, 0.0, 0.0])


# @eckart(x0, masses)
@com(masses)
def internal_to_cartesian(internal_coords):
    r1, r2, r3, r4 = internal_coords[:4]  # C-H distances
    symm_alpha = internal_coords[4:]
    alpha_ini = np.array([109.5 * np.pi / 180] * 5)
    alpha, stdev = from_symmetrized_alpha_to_alpha(symm_alpha, alpha_ini)

    cos_alpha12, cos_alpha13, cos_alpha14, cos_alpha23, cos_alpha24, cos_alpha34 = [
        jnp.cos(elem) for elem in alpha
    ]

    sin_alpha12 = jnp.sqrt(1 - cos_alpha12**2)
    sin_alpha13 = jnp.sqrt(1 - cos_alpha13**2)
    sin_alpha14 = jnp.sqrt(1 - cos_alpha14**2)
    sin_alpha23 = jnp.sqrt(1 - cos_alpha23**2)
    sin_alpha24 = jnp.sqrt(1 - cos_alpha24**2)

    xyz_C = jnp.array([0.0, 0.0, 0.0])
    xyz_H1 = r1 * jnp.array([0.0, 0.0, 1.0])
    xyz_H2 = r2 * jnp.array([sin_alpha12, 0.0, cos_alpha12])

    v12 = xyz_H1
    v23 = xyz_H2
    n2 = v12 / jnp.linalg.norm(v12)
    n3 = jnp.cross(v23, v12)
    n3 = n3 / jnp.linalg.norm(n3)
    n1 = jnp.cross(n2, n3)

    cos_a = jnp.sum(n2 * v23) / jnp.linalg.norm(v23)
    sin_a = jnp.sqrt(1 - cos_a**2)
    cos_phi = (cos_alpha23 - cos_alpha13 * cos_a) / (sin_alpha13 * sin_a)
    sin_phi = jnp.sqrt(1 - cos_phi**2)

    xyz_H3 = r3 * (
        cos_alpha13 * n2 + sin_alpha13 * cos_phi * n1 + sin_alpha13 * sin_phi * n3
    )

    v12 = xyz_H1
    v23 = xyz_H2
    n2 = v12 / jnp.linalg.norm(v12)
    n3 = jnp.cross(v12, v23)
    n3 = n3 / jnp.linalg.norm(n3)
    n1 = jnp.cross(n3, n2)

    cos_a = jnp.sum(n2 * v23) / jnp.linalg.norm(v23)
    sin_a = jnp.sqrt(1 - cos_a**2)
    cos_phi = (cos_alpha24 - cos_alpha14 * cos_a) / (sin_alpha14 * sin_a)
    sin_phi = jnp.sqrt(1 - cos_phi**2)

    xyz_H4 = r4 * (
        cos_alpha14 * n2 + sin_alpha14 * cos_phi * n1 + sin_alpha14 * sin_phi * n3
    )

    return jnp.array([xyz_C, xyz_H1, xyz_H2, xyz_H3, xyz_H4])


xyz = internal_to_cartesian(x0)
print("Reference internal coordinates:\n", x0)
print("Reference Cartesian coordinates:\n", xyz)

Reference internal coordinates:
 [1.08596384 1.08596384 1.08596384 1.08596384 0.         0.
 0.         0.         0.        ]
Reference Cartesian coordinates:
 [[ 1.53854207e-17 -1.37356992e-19  6.92534613e-18]
 [ 1.53854207e-17 -1.37356992e-19  1.08596384e+00]
 [ 1.02385652e+00 -1.37356992e-19 -3.61987945e-01]
 [-5.11928262e-01 -8.86685759e-01 -3.61987945e-01]
 [-5.11928262e-01  8.86685759e-01 -3.61987945e-01]]


Compute Taylor-series expansion of G-matrix

In [18]:
max_order = 2  # max total expansion order
deriv_ind = [
    elem
    for elem in itertools.product(*[range(0, max_order + 1) for _ in range(len(x0))])
    if sum(elem) <= max_order
]

func = lambda x: Gmat(x, masses, internal_to_cartesian) / G_to_invcm

def jacfwd(x0, ind):
    f = func
    for _ in range(sum(ind)):
        f = jax.jacfwd(f)
    i = sum([(i,) * o for i, o in enumerate(ind)], start=tuple())
    return f(x0)[:, :, *i]

coefs = np.array(
    [jacfwd(x0, ind) / np.prod(factorial(ind)) for ind in deriv_ind]
)

In [20]:
# Print second derivative wrt last symmetrized angular coordinate
# the Coriolis part of G-matrix is huge when using Eckart frame, why?

ind = deriv_ind.index((0, 0, 0, 0, 0, 0, 0, 0, 2))

for i in range(coefs[ind].shape[0]):
    for j in range(coefs[ind].shape[1]):
        print(i, j, np.round(coefs[ind][i, j], 6))

0 0 -0.0
0 1 0.013889
0 2 0.006944
0 3 0.006944
0 4 0.031328
0 5 0.0
0 6 -0.022382
0 7 -0.022382
0 8 -0.009592
0 9 -0.0
0 10 0.0
0 11 -0.0
0 12 0.0
0 13 -0.0
0 14 0.0
1 0 0.013889
1 1 -0.0
1 2 0.006944
1 3 0.006944
1 4 0.031328
1 5 0.0
1 6 0.022382
1 7 0.022382
1 8 -0.009592
1 9 0.0
1 10 -0.015826
1 11 0.0
1 12 0.0
1 13 -0.0
1 14 0.0
2 0 0.006944
2 1 0.006944
2 2 0.0
2 3 0.013889
2 4 0.031328
2 5 0.0
2 6 -0.022382
2 7 0.022382
2 8 0.009592
2 9 -0.005874
2 10 -0.00113
2 11 0.0
2 12 -0.0
2 13 -0.0
2 14 0.0
3 0 0.006944
3 1 0.006944
3 2 0.013889
3 3 0.0
3 4 0.031328
3 5 -0.0
3 6 0.022382
3 7 -0.022382
3 8 0.009592
3 9 0.005874
3 10 -0.00113
3 11 -0.0
3 12 -0.0
3 13 0.0
3 14 0.0
4 0 0.031328
4 1 0.031328
4 2 0.031328
4 3 0.031328
4 4 0.299081
4 5 0.0
4 6 0.0
4 7 0.0
4 8 -0.0
4 9 0.0
4 10 -0.020398
4 11 0.0
4 12 -0.0
4 13 0.0
4 14 -0.0
5 0 0.0
5 1 0.0
5 2 0.0
5 3 -0.0
5 4 0.0
5 5 -0.473267
5 6 0.0
5 7 -0.0
5 8 0.0
5 9 0.09108
5 10 -0.0
5 11 -0.17711
5 12 0.0
5 13 0.0
5 14 -0.0
6 0 -0.022382